In [4]:
from simple_wta import WTAProblem, random_wta_factory
from ortools.graph.python import min_cost_flow
import numpy as np

In [252]:
# lets do this in a more natural graph way
(n,m) = (3,3)
prob = random_wta_factory(n,m)
min_p = np.min(prob.p,axis=0)
solver = min_cost_flow.SimpleMinCostFlow()

L1 = np.array(range(n))
L2 = L1[-1]+1+np.array(range(m*n**2))
target_mask = [np.unravel_index(i,(n,m,n))[1] for i in range(len(L2))]
L3 = L2[-1]+1+np.array(range(n*m))
L4 = np.array([L3[-1]+1])
L = np.hstack((L1,L2,L3,L4))

# time to make a graph I guess
import networkx as nx
G = nx.DiGraph()
edges = []
# lets do this in a more natural graph way
(n,m) = (3,3)
prob = random_wta_factory(n,m)
min_p = np.min(prob.p,axis=0)
solver = min_cost_flow.SimpleMinCostFlow()

L1 = np.array(range(n))
L2 = L1[-1]+1+np.array(range(m*n**2))
target_mask = [np.unravel_index(i,(n,m,n))[1] for i in range(len(L2))]
L3 = L2[-1]+1+np.array(range(n*m))
L4 = np.array([L3[-1]+1])
L = np.hstack((L1,L2,L3,L4))

G.add_edges_from(edges)

start_nodes = []
end_nodes = []
capacities = []
unit_costs = []
for u,v in G.edges:
    data = G.get_edge_data(u,v)
    start_nodes.append(u)
    end_nodes.append(v)
    capacities.append(data['capacity'])
    unit_costs.append(data['unit_cost'])
supplies = np.zeros(len(L),dtype=int)
supplies[0:n] = 1
supplies[-1] = -n
solver.add_arcs_with_capacity_and_unit_cost(start_nodes,end_nodes,capacities,unit_costs)
for count, supply in enumerate(supplies):
    solver.set_node_supply(count,supply)

status = solver.solve()
status
edges
unit_costs[28:50]
# L2

[0.20056882717264726,
 0.10854409639812693,
 0.362201239863389,
 0.1551888271656602,
 0.06649224084969096,
 0.21268585110450366,
 0.18055277311474507,
 0.15327443602915958,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.4531048843897628,
 0.24521188533116073,
 0.13270408415182566,
 0.3184635079137466,
 0.1364489483990632]

In [253]:
#fuck-in-a
assignment = np.zeros(n,dtype=int)
foo = []
for i in range(solver.num_arcs()):
    if solver.tail(i) in range(n):
        if solver.flow(i)!=0:
            print(solver.flow(i))
            weapon = solver.tail(i)
            idx = solver.head(i)-n
            # target = np.unravel_index(idx,(n,m,n))
            print(np.unravel_index(idx,(n,m,n)))
            # assignment[weapon]=target
            foo.append((solver.tail(i),solver.head(i)-n))
# assignment
foo
# (start_nodes[0],end_nodes[0])
# start_nodes
# end_nodes

1
(0, 0, 0)
1
(1, 2, 2)
1
(2, 0, 1)


[(0, 0), (1, 17), (2, 19)]

In [315]:
import cvxpy as cvx
# lets do this in a more natural graph way
(n,m) = (10,10)
prob = random_wta_factory(n,m)
min_p = np.min(prob.p,axis=0)

# lets re-imagine this problem entirely

weapons = ["W%d"%(i) for i in range(n)]
target_nodes = ["T(%d,%d)"%(j,k) for j in range(m) for i in range(n)]

unit_costs = np.zeros(n*m*n)
for i in range(n):
    for j in range(m):
         for k in range(n):
            if k == 0:
                idx = np.ravel_multi_index((i,j,k),(n,m,n))
                unit_costs[idx] = prob.v[j]*(1-prob.p[i,j])
            else:
                unit_costs[idx] = prob.v[j]*((1-min_p[j])**k)*(1-prob.p[i,j])

# L12 connections represent flows from weapons to target nodes
L12_vars = np.array([cvx.Variable() for i in range(n*m*n)],dtype=cvx.Variable)
cost = np.dot(unit_costs,L12_vars)
# L23 connections represent flows from target nodes to sink node
L23_vars = np.array([cvx.Variable() for i in range(n*m)],dtype=cvx.Variable)

# capacity constraints (all link flows between zero and one)
const = []
const+=[0<=y for y in L12_vars]
const+=[1>=y for y in L12_vars]
const+=[0<=y for y in L23_vars]
const+=[1>=y for y in L23_vars]

# flow consistency constraints
for j in range(m):
    for k in range(n):
        idx1 = [np.ravel_multi_index((i,j,k),(n,m,n)) for i in range(n)]
        idx2 = np.ravel_multi_index((j,k),(m,n))
        # flow into node equals flow out for all target nodes
        const.append(np.sum(L12_vars[idx1])==L23_vars[idx2])

# demand constraint (flow into sink equals number of weapons)
const.append(np.sum(L23_vars)==n)

# contiguity constraints (see ahuja's paper)
for i in range(n):
    for j in range(m):
        for k in range(1,n):
            idx1 = np.ravel_multi_index((i,j,k),(n,m,n))
            idx2 = [np.ravel_multi_index((l,j,k-1),(n,m,n)) for l in range(n) if l!=i]
            const.append(L12_vars[idx1]==np.sum(L12_vars[idx2]))
# const
cvx_prob = cvx.Problem(cvx.Minimize(cost),const)
result = cvx_prob.solve()

In [326]:
# unpack the results
assignment = []
for idx, v in enumerate(L12_vars):
    # print(v)
    if v.value > 0:
        print(v.value)
    #     i,j,k = np.unravel_index(idx,(n,m,n))
    #     assignment.append((i,j,k))
        pass
cvx_prob.status


2.2919829939569942e-10
2.0812839742727065e-09
1.9067365023749042e-08
1.7156694650607233e-07
1.5441759809812005e-06
1.3897514287172674e-05
0.00012507769857563627
0.0011256992172411573
0.010131293025115892
0.09118163715609824
2.9806620970881677e-10
2.0498495627805585e-09
1.9130708160896357e-08
1.7178784651782287e-07
1.546513469030238e-06
1.3918202315206561e-05
0.00012526424019436633
0.0011273777424435853
0.010146400101303913
0.09131760049242425
2.1015352489888532e-10
2.083557630592471e-09
1.8995563427700472e-08
1.710087553583444e-07
1.5390638698136106e-06
1.3851593635822345e-05
0.00012466432435986308
0.001121978937652307
0.010097810420463076
0.09088029380257603
2.545452178899618e-10
2.097690066638559e-09
1.933921824425728e-08
1.7390289037338363e-07
1.565312187395422e-06
1.4087627653202159e-05
0.0001267888313859164
0.0011410993000203892
0.01026989388264257
0.09242904476132478
1.9385775095900321e-10
2.0930682567360464e-09
1.8999450830153875e-08
1.71123282922551e-07
1.5400150894511806e-06
1

'optimal'